In [1]:
from bs4 import BeautifulSoup

In [2]:
import pandas as pd

In [3]:
import re

In [7]:
import requests

In [8]:
url = "https://www.bfro.net/gdb/"
raw_html = requests.get(url).content

In [9]:
page = BeautifulSoup(raw_html, 'html.parser')

In [10]:
us_table = page.find_all('table')

In [11]:
table_content = us_table[0].find_all('table')[2:]

In [12]:
table_content[3].text

'Province# ofListingsMostRecentReportLastPostedAlberta433-20207-2018British Columbia13512-202012-2020Manitoba318-20121-2011New Brunswick812-20218-2021Province# ofListingsMostRecentReportLastPostedNova Scotia311-201610-2015Ontario688-20207-2020Quebec44-20081-2007Saskatchewan710-20084-2006Yukon55-20093-2009'

In [13]:
data = []
base = 'https://www.bfro.net'
for tables in table_content[1:]:
    rows = tables.find_all('tr')[1:]
    for row in rows:
        cols = row.find_all('td')
        state = cols[0].text.strip()
        a_tag = cols[0].find('a')
        link = base + a_tag['href'] if a_tag else None
        num_listings = cols[1].text.strip()
        most_recent_report = cols[2].text.strip()
        last_posted = cols[3].text.strip()
        data.append([state, num_listings, most_recent_report, last_posted,link])

In [14]:
df = pd.DataFrame(data, columns=["State/Province", "# of Listings", "Most Recent Report", "Last Posted","Link"])

In [15]:
df.head(-5)

,State/Province,# of Listings,Most Recent Report,Last Posted,Link
0,Alaska,22,12-2022,5-2022,https://www.bfro.net/GDB/state_listing.asp?sta...
1,Alabama,102,2-2023,2-2023,https://www.bfro.net/GDB/state_listing.asp?sta...
2,Arkansas,111,3-2023,9-2022,https://www.bfro.net/GDB/state_listing.asp?sta...
3,Arizona,85,6-2021,6-2021,https://www.bfro.net/GDB/state_listing.asp?sta...
4,California,463,10-2024,10-2024,https://www.bfro.net/GDB/state_listing.asp?sta...
...,...,...,...,...,...
75,Himalayan Region,10,5-2008,5-2008,https://www.bfro.net/GDB/show_county_reports.a...
76,Russia,5,5-1989,4-2015,https://www.bfro.net/GDB/show_county_reports.a...
77,Other Countries & Regions,6,11-2014,11-2011,https://www.bfro.net/GDB/show_county_reports.a...
78,Australia,1,6-2002,6-2002,https://www.bfro.net/GDB/show_county_reports.a...


# GoogleGeoAPI to find the coordinate

In [18]:
import time

In [13]:
import os
api_key = os.environ.get('GMAP_API')

In [14]:
placeNames = df['State/Province']

In [15]:
url = 'https://maps.googleapis.com/maps/api/geocode/json'
keys = {'address': 'Alaska', 'key': api_key}
r = requests.get(url,params=keys)
result_dic = r.json()

In [16]:
result_dic

{'results': [{'address_components': [{'long_name': 'Alaska',
     'short_name': 'AK',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'United States',
     'short_name': 'US',
     'types': ['country', 'political']}],
   'formatted_address': 'Alaska, USA',
   'geometry': {'bounds': {'northeast': {'lat': 71.439786,
      'lng': -129.9741939},
     'southwest': {'lat': 51.175092, 'lng': 172.347846}},
    'location': {'lat': 63.588753, 'lng': -154.4930619},
    'location_type': 'APPROXIMATE',
    'viewport': {'northeast': {'lat': 71.439786, 'lng': -129.9741939},
     'southwest': {'lat': 51.175092, 'lng': 172.347846}}},
   'place_id': 'ChIJG8CuwJzfAFQRNduKqSde27w',
   'types': ['administrative_area_level_1', 'political']}],
 'status': 'OK'}

In [28]:
geometry_df = []
#looping through the placeNames list
for place in placeNames:
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    keys = {'address': place, 'key': api_key}
    r = requests.get(url,params=keys)
    result_dic = r.json()
    if 'results' in result_dic and len(result_dic['results']) > 0:
        lat = result_dic['results'][0]['geometry']['location']['lat']
        long = result_dic['results'][0]['geometry']['location']['lng']
        coords = []
        coords.append(long)
        coords.append(lat)
        mygeometry = {'place': place, 'geometry.type': 'Point','geometry.coordinates':coords}
        geometry_df.append(mygeometry)
    else:
        print("No results found. Check your query or API key.")
    print(mygeometry)
    time.sleep(0.5)

{'place': 'Alaska', 'geometry.type': 'Point', 'geometry.coordinates': [-154.4930619, 63.588753]}
{'place': 'Alabama', 'geometry.type': 'Point', 'geometry.coordinates': [-86.902298, 32.3182314]}
{'place': 'Arkansas', 'geometry.type': 'Point', 'geometry.coordinates': [-91.8318334, 35.20105]}
{'place': 'Arizona', 'geometry.type': 'Point', 'geometry.coordinates': [-111.0937311, 34.0489281]}
{'place': 'California', 'geometry.type': 'Point', 'geometry.coordinates': [-119.4179324, 36.778261]}
{'place': 'Colorado', 'geometry.type': 'Point', 'geometry.coordinates': [-105.7820674, 39.5500507]}
{'place': 'Connecticut', 'geometry.type': 'Point', 'geometry.coordinates': [-73.087749, 41.6032207]}
{'place': 'Delaware', 'geometry.type': 'Point', 'geometry.coordinates': [-75.52766989999999, 38.9108325]}
{'place': 'Florida', 'geometry.type': 'Point', 'geometry.coordinates': [-81.5157535, 27.6648274]}
{'place': 'Georgia', 'geometry.type': 'Point', 'geometry.coordinates': [43.35689199999999, 42.315407]}
{

In [31]:
geometry = pd.DataFrame(geometry_df)

In [35]:
geometry.head()

,place,geometry.type,geometry.coordinates
0,Alaska,Point,"[-154.4930619, 63.588753]"
1,Alabama,Point,"[-86.902298, 32.3182314]"
2,Arkansas,Point,"[-91.8318334, 35.20105]"
3,Arizona,Point,"[-111.0937311, 34.0489281]"
4,California,Point,"[-119.4179324, 36.778261]"


In [33]:
geometry.to_csv('Bigfoot_geo.csv',index=False)

In [17]:
merged = df.merge(geometry, left_on='State/Province', right_on='place')

In [18]:
merged.head()

,State/Province,# of Listings,Most Recent Report,Last Posted,Link,place,geometry.type,geometry.coordinates
0,Alaska,22,12-2022,5-2022,https://www.bfro.net/GDB/state_listing.asp?sta...,Alaska,Point,"[-154.4930619, 63.588753]"
1,Alabama,102,2-2023,2-2023,https://www.bfro.net/GDB/state_listing.asp?sta...,Alabama,Point,"[-86.902298, 32.3182314]"
2,Arkansas,111,3-2023,9-2022,https://www.bfro.net/GDB/state_listing.asp?sta...,Arkansas,Point,"[-91.8318334, 35.20105]"
3,Arizona,85,6-2021,6-2021,https://www.bfro.net/GDB/state_listing.asp?sta...,Arizona,Point,"[-111.0937311, 34.0489281]"
4,California,463,10-2024,10-2024,https://www.bfro.net/GDB/state_listing.asp?sta...,California,Point,"[-119.4179324, 36.778261]"


In [19]:
merged['Lat']=merged['geometry.coordinates'].apply(lambda x:x[1])
merged['Lon']=merged['geometry.coordinates'].apply(lambda x:x[0])

In [20]:
merged.head()

,State/Province,# of Listings,Most Recent Report,Last Posted,Link,place,geometry.type,geometry.coordinates,Lat,Lon
0,Alaska,22,12-2022,5-2022,https://www.bfro.net/GDB/state_listing.asp?sta...,Alaska,Point,"[-154.4930619, 63.588753]",-,[
1,Alabama,102,2-2023,2-2023,https://www.bfro.net/GDB/state_listing.asp?sta...,Alabama,Point,"[-86.902298, 32.3182314]",-,[
2,Arkansas,111,3-2023,9-2022,https://www.bfro.net/GDB/state_listing.asp?sta...,Arkansas,Point,"[-91.8318334, 35.20105]",-,[
3,Arizona,85,6-2021,6-2021,https://www.bfro.net/GDB/state_listing.asp?sta...,Arizona,Point,"[-111.0937311, 34.0489281]",-,[
4,California,463,10-2024,10-2024,https://www.bfro.net/GDB/state_listing.asp?sta...,California,Point,"[-119.4179324, 36.778261]",-,[


In [21]:
merged.to_csv('geo_coord.csv')

# Scrape County level data for US

In [92]:
us_states = merged[:49]

In [106]:
data = []
base = 'https://www.bfro.net/GDB/'
for url in us_states['Link']:
    raw_html = requests.get(url).content
    html = BeautifulSoup(raw_html,'html.parser')
    counties = html.find_all('table')[3].find_all('table')[1].find_all('tr')[0]
    for c in counties:
        rows = counties.find_all('tr')[1:]
        for row in rows:
            cols = row.find_all('td')
            if len(cols) <4:
                continue
            county_tag = cols[0].find('a')
            if county_tag:
                county = county_tag.text
                link = base+county_tag.get('href')
            else:
                county = cols[0].text.strip()
                link = None
    
            num_listings = cols[1].text.strip()
            most_recent_report = cols[2].text.strip()
            last_posted = cols[3].text.strip()
            data.append([county, num_listings, most_recent_report, last_posted,link])

In [107]:
us_counties = pd.DataFrame(data,columns=["Counties", "# of Listings", "Most Recent Report", "Last Posted","Link"])
us_counties.head()

,Counties,# of Listings,Most Recent Report,Last Posted,Link
0,Aleutian Islands East,,,,None
1,Aleutian Islands West,,,,None
2,Anchorage,2,2-2004,11-2005,https://www.bfro.net/GDB/show_county_reports.a...
3,Angoon,,,,None
4,Barrow,,,,None


In [ ]:
data = []
base = 'https://www.bfro.net'
for tables in table_content[1:]:
    rows = tables.find_all('tr')[1:]
    for row in rows:
        cols = row.find_all('td')
        state = cols[0].text.strip()
        a_tag = cols[0].find('a')
        link = base + a_tag['href'] if a_tag else None
        num_listings = cols[1].text.strip()
        most_recent_report = cols[2].text.strip()
        last_posted = cols[3].text.strip()
        data.append([state, num_listings, most_recent_report, last_posted,link])

# scrape Class A/B 